In [139]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 

In [140]:
print_Debt = True

In [141]:
quantum_columns = str(export_from_RISKCUSTOM("""select *from "RISKACCESS"."quantumDebt" fetch first 1 rows only""").columns.tolist()).replace('\'','\"')[1:-1]
dates = ['31/08/24', '30/09/24']
dates_SQL = str([f'TO_DATE(\'{item}\', \'DD/MM/YY\')' for item in dates])[1:-1].replace('\"','')
dates_SQL

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  def export_from_RISKCUSTOM(query):


"TO_DATE('31/08/24', 'DD/MM/YY'), TO_DATE('30/09/24', 'DD/MM/YY')"

In [142]:
# quantum debt
query = f"""
with table_1 as
(select 
{quantum_columns},
--"reportDate","effectiveFrom","entity","dealNo","holding","counterparty","amount","eventCurrency","amountUSD","actionDate","dealClass1","dealClass2","instrumentOwner","dealSet","facility", 
MAX("effectiveFrom") OVER (partition BY "reportDate") as "max_effectiveFrom"
from "RISKACCESS"."quantumDebt" 
where "reportDate" in ({dates_SQL})
--fetch first 10 rows only
)
select * from table_1
where "effectiveFrom" = table_1."max_effectiveFrom"
--fetch first 10 rows only
"""
data_Debt_export = export_from_RISKCUSTOM(query)
data_Debt_export.head(2)

,##batch_id,##deleted,##inserted,##origin_name,flag,actionDate,amount,eventCurrency,amountUSD,currentRate,...,debtRelated,reportDate,eventNumber,entitySAPBUCode,counterpartySAPID,counterpartySWIFTBICCode,effectiveFrom,xlsDate,loadEventId,max_effectiveFrom
0,41482288@WHPROD,N,2024-10-28 23:15:32,QUANTUM,-,2024-09-06,-7000000.0,USD,-7000000.0,6.3,...,Y,2024-08-31,241107,E920,1000140127,None,2024-10-28,2024-10-28 23:14:37,278556,2024-10-28
1,41482288@WHPROD,N,2024-10-28 23:15:32,QUANTUM,-,2024-09-06,-5000000.0,USD,-5000000.0,6.3,...,Y,2024-08-31,241108,E920,1000140127,None,2024-10-28,2024-10-28 23:14:37,278556,2024-10-28


In [143]:
assert len(data_Debt_export[data_Debt_export.entity.isna()]) == 0 # data quality
data_Debt_work = data_Debt_export.copy() # copy data
# merge
data_Debt_work['entity_code'] = merge_Mapping(data_Debt_work, col='entity')
data_Debt_work.loc[data_Debt_work.entity_code=='External', 'entity_code'] = data_Debt_work.loc[data_Debt_work.entity_code=='External', 'entity']
data_Debt_work['Entity_group'] = merge_SalesUnits(data_Debt_work, col='entity_code', merge_col='ocpSegment')

# Выполнение цикла для рассчета amount, amountUSD и присваивания значения counterparty для каждого DealSet

# Выбор нужного разреза данных
deals_data_Bonds = data_Debt_work.loc[data_Debt_work.dealSet.str.contains('Bond'), ['entity_code', 'Entity_group', 'counterparty','amount', 'amountUSD', 'instrumentOwner', 'dealSet','reportDate']] # Фильтрация разреза по значениям в DealSet, которые содержат слово Bond

Bonds_list = deals_data_Bonds.dealSet.unique()

deals_data_Bonds_work = pd.DataFrame()
for Bond in Bonds_list:
    try:
        Bond_data = deals_data_Bonds[deals_data_Bonds.dealSet==Bond] # Отбор данных одного Bond
        Bond_data_traded = Bond_data[Bond_data.instrumentOwner=='EC_TRADED_BONDS']
        Bond_data_issued = Bond_data[Bond_data.instrumentOwner=='EC_ISSUED_BONDS']
        if len(Bond_data_issued)>1:
            Bond_data_issued['index_copy'] = Bond_data_issued.index # copy index
            # groupby
            Bond_data_issued_pivot = Bond_data_issued.groupby(as_index=False, dropna=False, by=['entity_code',	'Entity_group',	'counterparty', 'instrumentOwner',	'dealSet',	'reportDate']).agg({'amount':'sum', 'amountUSD':'sum', 'index_copy':'min'})
            Bond_data_issued_pivot.index = [int(Bond_data_issued_pivot.index_copy.item())] # index on min in group
            Bond_data_issued_pivot.drop(columns='index_copy') # drop index copy col
            Bond_data = pd.concat([Bond_data_traded, Bond_data_issued_pivot])
        assert len(Bond_data_issued) != 0

        columns_list = ['amount', 'amountUSD']
        for column in columns_list:
            Traded_data_sum = float(Bond_data.loc[Bond_data.instrumentOwner=='EC_TRADED_BONDS', column].sum())
            Issued_data = float(Bond_data.loc[Bond_data.instrumentOwner=='EC_ISSUED_BONDS', column].item())
            # Внесение нового значения amount в issued bonds
            Bond_data.loc[Bond_data['instrumentOwner'] =="EC_ISSUED_BONDS", column] = Traded_data_sum + Issued_data 
        # Внесение нового значения counterparty в traded bonds
        Issied_data_entity = Bond_data.loc[Bond_data.instrumentOwner=='EC_ISSUED_BONDS', 'entity_code'].squeeze()
        Bond_data.loc[Bond_data['instrumentOwner'] =="EC_TRADED_BONDS", 'counterparty'] = Issied_data_entity
        
        deals_data_Bonds_work = pd.concat([deals_data_Bonds_work, Bond_data])
        # deals_data_Bonds[deals_data_Bonds.dealSet==Bond] = Bond_data
    except:
        continue

index_list = deals_data_Bonds_work.index.to_list()
data_Debt_work.loc[(data_Debt_work.dealSet.str.contains('Bond'))&(data_Debt_work.index.isin(index_list)), ['entity_code', 'Entity_group', 'counterparty','amount', 'amountUSD', 'instrumentOwner', 'dealSet','reportDate']] = deals_data_Bonds_work
data_Debt_work.loc[(data_Debt_work.dealSet.str.contains('Bond'))&(~data_Debt_work.index.isin(index_list)), 'entity_code'] = np.nan
data_Debt_work = data_Debt_work[~data_Debt_work.entity_code.isna()]
# Запись выполненного цикла в в основные данные

data_Debt_work = Period(data_Debt_work, day_for_count='reportDate', col_with_date='actionDate', day_is_col=True).reset_index(drop=True)

data_Debt_work.loc[data_Debt_work.counterparty.str.contains('(EC)'), 'counterparty_name'] = data_Debt_work.loc[data_Debt_work.counterparty.str.contains('(EC)'), 'counterparty'].str[:-5] 
# Обрезка " (EC)" в каждой строке

assert len(data_Debt_export[data_Debt_export.counterparty.isna()])==0
data_Debt_work['cty_Code'] = merge_Mapping(data_Debt_work, col='counterparty_name')
data_Debt_work['Counterparty_Group'] = merge_SalesUnits(data_Debt_work, col='cty_Code', merge_col='ocpSegment')
# Merge counterparty с Mapping и SalesUnits

data_Debt_work['Source'] = 'Quantum'

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  def export_from_RISKCUSTOM(query):
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['TRADING' 'TRADING' 'TRADING' ... 'SGC' 'SGC' 'SGC']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_940\1178440806.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [144]:
# instrumentOwner_list = ['EC_ISSUED_BONDS', 'EC_TRADED_BONDS']
# deals_data_Bonds[(deals_data_Bonds.dealSet==Bonds_list[0])&(deals_data_Bonds.reportDate==dates[0])&(deals_data_Bonds.instrumentOwner==instrumentOwner_list[1])]
# data_Debt_work.loc[(data_Debt_work.dealSet==Bonds_list[0])&(data_Debt_work.reportDate==dates[0])&(data_Debt_work.instrumentOwner==instrumentOwner_list[1]), ['entity_code','Entity_group','counterparty','amount','amountUSD','instrumentOwner','dealSet','reportDate']]
# data_Debt_export.loc[(data_Debt_export.instrumentOwner=='EC_TRADED_BONDS')&(data_Debt_export.dealSet=='SUEK EuBond 2021 $500m B'), ['entity','counterparty','instrumentOwner','dealSet','reportDate']]

Деривативы:

In [145]:
query = f"""select * 
from "RISKACCESS"."quantumForwards" 
where "positionDate" in ({dates_SQL})"""
data_Forwards_export = export_from_RISKCUSTOM(query)

query = f"""select * 
from "RISKACCESS"."quantumSwaps" 
where "positionDate" in ({dates_SQL})"""
data_Swaps_export = export_from_RISKCUSTOM(query)

data_Deriv_qu = pd.concat([data_Forwards_export,data_Swaps_export])
data_Deriv_qu.head(2)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  def export_from_RISKCUSTOM(query):
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_940\367103458.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_Deriv_qu = pd.concat([data_Forwards_export,data_Swaps_export])
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_940\367103458.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns wh

,##batch_id,##deleted,##inserted,##origin_name,adjustedProfitLoss,baseCurrency,baseType,checkSettlementDateAlt,contractRate,counterparty,...,recMargin,recPrincipalFreq,recRateBasisCode,recRateBasisFreq,recRateBasisName,recRateFormula,recRatesetFreq,shortInterestRate,shortNPVOfAdjustedProfitLoss,startdate
0,40656537@WHPROD,N,2024-10-01 01:00:17,QUANTUM,344.36,USD,CONTRACT,Y,1.367691,Western Alliance US (EC),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,40656537@WHPROD,N,2024-10-01 01:00:17,QUANTUM,-2003.71,USD,CONTRACT,Y,1.359600,Western Alliance US (EC),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [146]:
# проверка вхождения в debt
Deriv_dealNo = data_Deriv_qu.dealNo.unique().tolist()
Debt_dealNo = data_Debt_export.dealNo.unique().tolist()
Deriv_dealNo_new = [x for x in Deriv_dealNo if x not in Debt_dealNo]
data_Deriv_qu_work = data_Deriv_qu[data_Deriv_qu.dealNo.isin(Deriv_dealNo_new)]
# merge
data_Deriv_qu_work['Entity_group'] = merge_SalesUnits(data_Deriv_qu_work, col='entity', merge_col='ocpSegment')
# melt data
melt_data = pd.melt(data_Deriv_qu_work, id_vars=['positionDate','holdingEntity','Entity_group','entity','counterparty','dealNo','payFaceValue','recFaceValue','maturityDate'], value_vars=['payFXCurrency','recFXCurrency'], value_name='Currency').sort_values('dealNo').rename(columns={'recFaceValue':'amount'})
melt_data.loc[melt_data.variable=='payFXCurrency', 'amount'] = melt_data.loc[melt_data.variable=='payFXCurrency', 'payFaceValue'] * -1
melt_data = melt_data.drop(columns=['payFaceValue','variable']).reset_index(drop=True)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  def export_from_RISKCUSTOM(query):
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES'
 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES'
 'NAMSALES' 'SAMSALES' 'SAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES'
 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES'
 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES'
 'NAMSALES' 'NAMSALES']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  
C:\Users\KlimovaAnnaA

Долг OCP

In [147]:
query = f"""
select * 
from xxwh."unifiedOcp"
where "sourceTable" in ('quantumDebt_bonds','quantumDebt')
and "reportDate" in ({dates_SQL})
--fetch first 10 rows only
"""
data_OCP_export = export_from_WHWEEK(query)
data_OCP_export.head(2)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:176: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  def export_from_WHWEEK(query):


,##batch_id,##deleted,##inserted,##origin_name,reportDate,rowType,sourceSystem,sourceTable,recordId,balanceItem,...,timeBucket,notionalCurrency,volumeNotionalCurrency,effectiveCurrency,effectiveCurrencyRate,volumeEffectiveCurrency,usdRate,volumeUsd,salesUnitsId,accountSubType
0,41448664@WHWEEK,N,2024-10-28 10:42:41,WHFLEX,2024-08-31,M,QUANTUM,quantumDebt,1466,On-balance,...,1M-3M,RUB,18416670.94,RUB,1,18416670.94,0.011054,203577.183075,1100,None
1,41448664@WHWEEK,N,2024-10-28 10:42:41,WHFLEX,2024-08-31,M,QUANTUM,quantumDebt,1466,On-balance,...,1M-3M,USD,-244561.06,USD,1,-244561.06,1.000000,-244561.060000,1100,None


In [148]:
### Compairing Debt
# rename
rename_dict = {'ocpSegment':'Entity_group','effectiveCurrency':'eventCurrency','salesUnitsId':'entity_code','volumeEffectiveCurrency':'amount_OCP','volumeUsd':'amountUSD_OCP','counterpartySegment':'Counterparty_Group','counterpartyName':'counterparty','introgroupCounterpartyCode':'cty_Code','recordId':'dealNo'}
data_OCP_work = data_OCP_export.copy()
data_OCP_work = data_OCP_work.rename(columns=rename_dict)
data_OCP_work.cty_Code = data_OCP_work.cty_Code.fillna('External')
data_Debt_work.holding = data_Debt_work.holding.replace({'EuroChem':'EUROCHEM'})
# first compairing
Debt_index_list = ['reportDate', 'holding', 'Entity_group', 'entity_code', 'eventCurrency']
Debt_first = data_Debt_work.groupby(as_index=False,dropna=False, by=Debt_index_list)[['amount', 'amountUSD']].agg(sum)
Debt_first.holding = Debt_first.holding.replace({'EuroChem':'EUROCHEM'})
OCP_first = data_OCP_work.groupby(as_index=False, dropna=False, by=Debt_index_list)[['amount_OCP','amountUSD_OCP']].agg(sum)
# merge
First_merge = Debt_first.merge(OCP_first, how='outer', on=Debt_index_list)
First_merge = is_approximately_equal_for_cols(First_merge, 'amount', 'amount_OCP')
First_merge_not_equal = First_merge[First_merge.amount!=First_merge.amount_OCP]

# second compairing
First_merge_not_equal = concat_columns(First_merge_not_equal, columns=Debt_index_list)
concat_list = First_merge_not_equal.concat_columns.tolist()
data_Debt_work = concat_columns(data_Debt_work, columns=Debt_index_list)
data_OCP_work = concat_columns(data_OCP_work, columns=Debt_index_list)

Debt_index_list = ['reportDate', 'holding', 'Entity_group', 'entity_code', 'eventCurrency', 'Counterparty_Group','counterparty','cty_Code','dealNo']
data_Debt_work.dealNo,data_OCP_work.dealNo = data_Debt_work.dealNo.astype(int),data_OCP_work.dealNo.astype(int)
Debt_second = data_Debt_work[data_Debt_work.concat_columns.isin(concat_list)].groupby(as_index=False,dropna=False, by=Debt_index_list)[['amount', 'amountUSD']].agg(sum)
OCP_second = data_OCP_work[data_OCP_work.concat_columns.isin(concat_list)].groupby(as_index=False, dropna=False, by=Debt_index_list)[['amount_OCP','amountUSD_OCP']].agg(sum)
# merge
Second_merge = Debt_second.merge(OCP_second, how='outer', on=Debt_index_list)

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_940\886512111.py:10: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  Debt_first = data_Debt_work.groupby(as_index=False,dropna=False, by=Debt_index_list)[['amount', 'amountUSD']].agg(sum)
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_940\886512111.py:12: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  OCP_first = data_OCP_work.groupby(as_index=False, dropna=False, by=Debt_index_list)[['amount_OCP','amountUSD_OCP']].agg(sum)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:98: SettingWithCopyWarning: 
A value is trying to be set on a co

In [149]:
# если чего-то нет в amount, проверить есть ли тот же объем с обратным котрагентом
Second_merge['abs_amount_OCP'] = Second_merge.amount_OCP.abs()
Second_merge['index_copy'] = Second_merge.index
Second_merge = concat_columns(Second_merge, ['dealNo', 'eventCurrency','abs_amount_OCP', 'entity_code','cty_Code']).rename(columns={'concat_columns':'E_C'})
data_amount_isna = Second_merge[Second_merge.amount.isna()]
data_amount_isna = concat_columns(data_amount_isna, ['dealNo', 'eventCurrency', 'abs_amount_OCP', 'cty_Code', 'entity_code']).rename(columns={'concat_columns':'C_E'})
drop_index = data_amount_isna[data_amount_isna.C_E.isin(Second_merge.E_C.tolist())].index_copy.tolist() # берем те строки, лля которых есть обратные операции
Second_merge_not_duples = Second_merge[~Second_merge.index_copy.isin(drop_index)] # удаляем обатные операции
# сравниваем значения на равенство
Second_merge_not_duples.amount = Second_merge_not_duples.amount.fillna(0)
Second_merge_not_duples = is_approximately_equal_for_cols(Second_merge_not_duples, 'amount', 'amount_OCP')
# поделить суммы и если КФ равен 2, то проигнорировать ее
Second_merge_not_duples['CF_del_amounts'] = Second_merge_not_duples.amount_OCP/Second_merge_not_duples.amount
Second_merge_not_duples.loc[Second_merge_not_duples.CF_del_amounts==2, 'Diff__more_0.4'] = False
Second_merge_not_duples = Second_merge_not_duples.drop(columns=['index_copy','E_C'])

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_940\3226732605.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Second_merge_not_duples.amount = Second_merge_not_duples.amount.fillna(0)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:184: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [150]:
# точно ли там где amount равен нулю, в исходных данных нет информации по данному номеру сделки
delano_list = Second_merge_not_duples.loc[Second_merge_not_duples['Diff__more_0.4']==True, 'dealNo'].tolist()
manual_check_dealno_list = list(set(data_Debt_work.dealNo.tolist()).intersection(set(delano_list)))

Деривативы OCP

In [151]:
query = f"""
select * 
from xxwh."unifiedOcp"
where "sourceTable" in ('quantumForwards', 'quantumSwaps')
and "reportDate" in ({dates_SQL})
--fetch first 10 rows only
"""
# quantumSwaps
data_Deriv_OCP = export_from_WHWEEK(query)
data_Deriv_OCP.head(2)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:176: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  def export_from_WHWEEK(query):


,##batch_id,##deleted,##inserted,##origin_name,reportDate,rowType,sourceSystem,sourceTable,recordId,balanceItem,...,timeBucket,notionalCurrency,volumeNotionalCurrency,effectiveCurrency,effectiveCurrencyRate,volumeEffectiveCurrency,usdRate,volumeUsd,salesUnitsId,accountSubType
0,40656537@WHPROD,N,2024-10-01 01:01:20,WHFLEX,2024-08-31,D,QUANTUM,quantumForwards,8940,Off-balance,...,1M-3M,USD,-219764.12,USD,1,-219764.12,1.00000,-219764.120000,USBFC,None
1,40656537@WHPROD,N,2024-10-01 01:01:20,WHFLEX,2024-08-31,D,QUANTUM,quantumForwards,8940,Off-balance,...,1M-3M,CAD,300000.00,CAD,1,300000.00,0.74118,222353.987548,USBFC,None


In [152]:
# проверка вхождения в debt
Deriv_dealNo = data_Deriv_OCP.recordId.unique().tolist()
Debt_dealNo = data_OCP_export.recordId.unique().tolist()
Deriv_dealNo_new = [x for x in Deriv_dealNo if x not in Debt_dealNo]
data_Deriv_OCP_work = data_Deriv_OCP[data_Deriv_OCP.recordId.isin(Deriv_dealNo_new)]
data_Deriv_OCP_work.shape, data_Deriv_OCP.shape

((80, 40), (86, 40))

In [153]:
data_Deriv_OCP_work = data_Deriv_OCP_work[['reportDate','holding','ocpSegment','buCode','salesUnitsId','counterpartyName','recordId','effectiveCurrency','volumeEffectiveCurrency','maturityDateTo']]
# merge
melt_data_cols = ['positionDate','holdingEntity','Entity_group','entity','counterparty','dealNo','Currency'] # index col
melt_data.holdingEntity = melt_data.holdingEntity.replace({'EuroChem':'EUROCHEM'}) # replace
# rename
rename_dict = {'reportDate':'positionDate','holding':'holdingEntity','ocpSegment':'Entity_group','buCode':'entity','counterpartyName':'counterparty','recordId':'dealNo','effectiveCurrency':'Currency','volumeEffectiveCurrency':'amount_OCP','maturityDateTo':'maturityDateTo_OCP'} 
data_Deriv_OCP_work = data_Deriv_OCP_work.rename(columns=rename_dict)
# dealNo int
melt_data['dealNo'] = melt_data['dealNo'].astype(int)
data_Deriv_OCP_work['dealNo'] = data_Deriv_OCP_work['dealNo'].astype(int)

Deriv_merge = melt_data.merge(data_Deriv_OCP_work, how='outer', on=melt_data_cols)
# проверка значений на равенство
Deriv_merge.amount = Deriv_merge.amount.fillna(0)
Deriv_merge = is_approximately_equal_for_cols(Deriv_merge, 'amount', 'amount_OCP')

In [154]:
### to excel
if print_Debt==True:
    Output_file = f'{str(date.today())}_quDebt_vs_unOCP.xlsx'
    with pd.ExcelWriter(Output_file, engine='openpyxl') as writer:
        pd.DataFrame({'manual_check_dealNo':manual_check_dealno_list}, index=range(len(manual_check_dealno_list))).to_excel(writer, index=False, sheet_name='manual_check')
        # First_merge.to_excel(writer, index=False, sheet_name='compare_debt_BU')
        Second_merge_not_duples.to_excel(writer, index=False, sheet_name='compare_debt_cpty')
        Deriv_merge.to_excel(writer, index=False, sheet_name='compare_deriv')
        data_Debt_work.to_excel(writer, index=False, sheet_name='quDebt')
        data_OCP_export.to_excel(writer, index=False, sheet_name='unOCP')
